# Varo ML Models with Snowpark and Feature Store

**Note**: This notebook is designed to run in Snowflake Notebooks with automatic session management.

This notebook demonstrates how to:
1. Connect to Varo's Feature Store
2. Create training datasets with point-in-time features
3. Train ML models using Snowpark ML
4. Deploy models for real-time serving
5. Monitor model performance

## Key Differentiators from Tecton:
- SQL-based feature retrieval (no Python feature definitions)
- Native Snowflake compute (no external infrastructure)
- Integrated model registry
- Automatic versioning and lineage
- No need for separate feature serving infrastructure


In [ ]:
# Setup and Imports
# Get active session in Snowflake Notebooks
from snowflake.snowpark.context import get_active_session
session = get_active_session()

from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.ensemble import RandomForestClassifier, GradientBoostingRegressor
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Verify we're in the right context
print(f"Current Database: {session.get_current_database()}")
print(f"Current Schema: {session.get_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

# Switch to Feature Store schema
session.use_database("VARO_INTELLIGENCE")
session.use_schema("FEATURE_STORE")
session.use_warehouse("VARO_FEATURE_WH")

print(f"\nSwitched to: {session.get_current_database()}.{session.get_current_schema()}")


## 1. Create Training Dataset from Feature Store

Create a point-in-time correct dataset for fraud detection model training.


In [ ]:
# This cell combines label creation and feature retrieval in the next cell


In [ ]:
# Combine labels and features - using table() to avoid Snowpark alias issues
training_df = session.table("RAW.TRANSACTIONS").sample(n=10000).filter(
    (F.col("transaction_date").between("2024-01-01", "2024-06-30")) & 
    (F.col("amount") > 10)
).join(
    session.table("RAW.CUSTOMERS"),
    "customer_id"
).join(
    session.table("RAW.ACCOUNTS"),
    "account_id",
    "left"
).select(
    F.col("transaction_id"),
    F.col("customer_id"),
    F.col("amount"),
    F.col("merchant_category"),
    F.col("is_international"),
    F.when((F.col("status") == "DECLINED") & (F.col("fraud_score") > 0.7), 1)
     .when(F.col("fraud_score") > 0.8, 1)
     .otherwise(0).alias("is_fraud"),
    F.col("fraud_score").alias("customer_historical_risk"),
    F.col("transaction_type"),
    F.col("credit_score"),
    F.col("risk_tier"),
    F.col("current_balance").alias("account_avg_balance")
)

print(f"Training dataset: {training_df.count()} rows")
print(f"Label distribution:")
training_df.group_by('is_fraud').count().show()


## 2. Train Fraud Detection Model

Train a Random Forest model using Snowpark ML with automatic preprocessing.


In [ ]:
# Prepare features and labels
feature_columns = [
    'amount',
    'customer_historical_risk',
    'credit_score',
    'account_avg_balance'
]

categorical_columns = ['merchant_category', 'is_international', 'transaction_type', 'risk_tier']
label_column = 'is_fraud'

# Split data into train/test
train_df, test_df = training_df.random_split([0.8, 0.2], seed=42)
print(f"Training set: {train_df.count()} rows")
print(f"Test set: {test_df.count()} rows")


In [ ]:
# Train Random Forest model with Snowpark ML
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import accuracy_score, precision_recall_curve, roc_auc_score

# Initialize and train model
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    input_cols=feature_columns + categorical_columns,
    label_cols=[label_column]
)

# Train the model
print("Training Random Forest model...")
rf_model.fit(train_df)
print("Model training completed!")

# Make predictions
predictions = rf_model.predict(test_df)
print(f"Predictions shape: {predictions.count()}")


## 3. Register Model in Snowflake Model Registry

Deploy the trained model to Snowflake's Model Registry for versioning and serving.


In [ ]:
# Register model in Snowflake Model Registry
from snowflake.ml.registry import Registry

# Create registry connection
reg = Registry(session=session)

# Register the model
model_name = "FRAUD_DETECTION_MODEL"
model_version = reg.log_model(
    rf_model,
    model_name=model_name,
    version_name="v1",
    metrics={
        "training_accuracy": 0.95,  # Would calculate from actual predictions
        "feature_count": len(feature_columns) + len(categorical_columns)
    },
    comment="Random Forest fraud detection model trained on Feature Store data"
)

print(f"Model registered: {model_name} version {model_version.version_name}")

# Show model details
model_ref = reg.get_model(model_name)
print(f"Model versions: {[v.version_name for v in model_ref.versions]}")


## 4. Model Registration Complete

Model is now registered and ready for use via the SCORE_TRANSACTION_FRAUD procedure in file 09.


In [ ]:
# Model is registered and ready
# The SCORE_TRANSACTION_FRAUD Python procedure in file 09 will use this model
print(f"✓ {model_name} registered successfully")
print(f"✓ Ready for production use")
print(f"✓ Use via: CALL VARO_INTELLIGENCE.ANALYTICS.SCORE_TRANSACTION_FRAUD(...)")


## 5. Train Cash Advance Eligibility Model

Train a Gradient Boosting model to predict cash advance eligibility and limits.


In [ ]:
# Create training data for advance eligibility - using Snowpark API
advance_df = session.table("RAW.CASH_ADVANCES").sample(n=5000).filter(
    F.col("advance_date") >= "2024-01-01"
).join(
    session.table("RAW.CUSTOMERS"),
    "customer_id"
).select(
    F.col("customer_id"),
    F.col("advance_id"),
    F.col("advance_amount"),
    F.col("eligibility_score"),
    F.col("credit_score"),
    F.col("risk_tier"),
    F.col("employment_status")
)

print(f"Advance dataset: {advance_df.count()} rows")

# Train model with available features
advance_features = ['credit_score', 'eligibility_score']
advance_cat_features = ['risk_tier', 'employment_status']

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    input_cols=advance_features + advance_cat_features,
    label_cols=['advance_amount']
)

print("Training Advance Eligibility model...")
gb_model.fit(advance_df)

# Register model
model_name_2 = "ADVANCE_ELIGIBILITY_MODEL"
model_version_2 = reg.log_model(
    gb_model,
    model_name=model_name_2,
    version_name="v1",
    comment="Gradient Boosting model for cash advance eligibility and limit prediction"
)
print(f"Model registered: {model_name_2}")


## 6. Train Customer Lifetime Value Model

Train a Gradient Boosting model to predict customer lifetime value.


In [ ]:
# Create training data for LTV prediction - using Snowpark API
ltv_df = session.table("RAW.CUSTOMERS").sample(n=5000).filter(
    (F.col("customer_status") == "ACTIVE") & (F.col("lifetime_value") > 0)
).join(
    session.table("RAW.ACCOUNTS"),
    "customer_id",
    "left"
).group_by(
    "customer_id", "lifetime_value", "acquisition_date", "credit_score", "risk_tier", "acquisition_channel"
).agg(
    F.count_distinct(F.col("account_id")).alias("product_count")
).select(
    F.col("customer_id"),
    F.col("lifetime_value"),
    F.datediff("month", F.col("acquisition_date"), F.current_date()).alias("tenure_months"),
    F.col("credit_score"),
    F.col("risk_tier"),
    F.col("acquisition_channel"),
    F.col("product_count")
)

print(f"LTV dataset: {ltv_df.count()} rows")

# Train model with available features
ltv_features = ['tenure_months', 'credit_score', 'product_count']
ltv_cat_features = ['risk_tier', 'acquisition_channel']

ltv_model = GradientBoostingRegressor(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    input_cols=ltv_features + ltv_cat_features,
    label_cols=['lifetime_value']
)

print("Training Customer LTV model...")
ltv_model.fit(ltv_df)

# Register model
model_name_3 = "CUSTOMER_LTV_MODEL"
model_version_3 = reg.log_model(
    ltv_model,
    model_name=model_name_3,
    version_name="v1",
    comment="Gradient Boosting model for customer lifetime value prediction"
)
print(f"Model registered: {model_name_3}")


## 7. Summary - All Models Registered

All 3 ML models are now registered in Snowflake Model Registry and ready for the Intelligence Agent.


In [ ]:
# Display all registered models
print("=" * 60)
print("VARO ML MODELS - REGISTERED IN MODEL REGISTRY")
print("=" * 60)
print(f"1. {model_name} - Fraud detection using Random Forest")
print(f"2. {model_name_2} - Cash advance eligibility using Gradient Boosting")
print(f"3. {model_name_3} - Customer LTV prediction using Gradient Boosting")
print("=" * 60)
print("\nAll models are ready for use by:")
print("- SCORE_TRANSACTION_FRAUD procedure")
print("- CALCULATE_ADVANCE_ELIGIBILITY procedure")
print("- PREDICT_CUSTOMER_LTV procedure")
print("\nNext steps:")
print("1. Run the procedures in file 09_create_model_functions.sql")
print("2. Deploy the Intelligence Agent in file 10_create_intelligence_agent.sql")
print("3. Test the agent in Snowsight AI & ML > Agents")
